In [ ]:
#Implementation of Jacobi
function jacobiIteration(A, b, x; iterMax=100)
    """Implements the Jacobi iteration.
    A is the coefficient matrix.
    b is the constant matrix.
    x is an initial guess."""

    D = Diagonal(A)
    f = D\b
    G = D\(D - A)
    for j=1:iterMax
        x = G*x + f
    end
    return x, maximum(abs.(eigvals(G)))
end

#Implementation of Gauss-Seidel

function gsIteration(A, b, x; iterMax = 100)
    """Implements the GS iteration.
    A is the coefficient matrix.
    b is the constant matrix.
    x is an initial guess."""
    DE = tril(A)
    F = -triu(A,1)
    G = DE\F
    f = DE\b
    for j=1:iterMax
        x = G*x + f
    end
    return x, maximum(abs.(eigvals(G)))
end

using LinearAlgebra
function generateProblem(n)
    A = randn(n,n)  + sqrt(n)*I
    x = randn(n)
    b = A*x
    return A, b, x
end


In [2]:
printstyled("EXAMPLE 1: n=10\n", color=:red)
A, b, x = generateProblem(10)
z_j, ρ_j = jacobiIteration(A,b,x)
z_gs, ρ_gs = gsIteration(A,b,x)
println("Jacobi Iteration")
println("\tAbsolute Error: $(norm(x-z_j))")
println("\tSpect. Radius: $ρ_j")
println("Gauss Seidel Iteration")
println("\tAbsolute Error: $(norm(x-z_gs))")
println("\tSpect. Radius: $ρ_gs\n\n")

EXAMPLE 1: n=10
Jacobi Iteration
	Absolute Error: 1.3520697848899533e-15
	Spect. Radius: 0.863490532954029
Gauss Seidel Iteration
	Absolute Error: 1.6793843546254288e-15
	Spect. Radius: 0.749387622986436




In [3]:
printstyled(
"EXAMPLE 2: n=50\n", color=:red)
A, b, x = generateProblem(50)
z_j, ρ_j = jacobiIteration(A,b,x)
z_gs, ρ_gs = gsIteration(A,b,x)
println("Jacobi Iteration")
println("\tAbsolute Error: $(norm(x-z_j))")
println("\tSpect. Radius: $ρ_j")
println("Gauss Seidel Iteration")
println("\tAbsolute Error: $(norm(x-z_gs))")
println("\tSpect. Radius: $ρ_gs\n\n")

printstyled(
"EXAMPLE 3: n=100\n", color=:red)
A, b, x = generateProblem(100)
z_j, ρ_j = jacobiIteration(A,b,x)
z_gs, ρ_gs = gsIteration(A,b,x)
println("Jacobi Iteration")
println("\tAbsolute Error: $(norm(x-z_j))")
println("\tSpect. Radius: $ρ_j")
println("Gauss Seidel Iteration")
println("\tAbsolute Error: $(norm(x-z_gs))")
println("\tSpect. Radius: $ρ_gs")


EXAMPLE 2: n=50
Jacobi Iteration
	Absolute Error: 1.0854189282875838e-11
	Spect. Radius: 1.087391060471747
Gauss Seidel Iteration
	Absolute Error: 1.2516341276417508e-9
	Spect. Radius: 1.1243980836661103


EXAMPLE 3: n=100
Jacobi Iteration
	Absolute Error: 1.127348526579172e-14
	Spect. Radius: 1.005846794439449
Gauss Seidel Iteration
	Absolute Error: 6.779188414146492e-13
	Spect. Radius: 1.0526951996953056


In [4]:
#Kaczmarz Update

function kaczmarzUpdate(b,a,x)
    """
    The Kaczmarz update, where b is a scalar,
    a is vector, and x is the current iterate.
    Returns the next iterate.
    """
    return x + ((b - dot(a,x))/dot(a,a))*a
end


#Cyclical Kaczmarz
function cyclicalKaczmarz(b, A, x;maxIter=5)
    """
    Estimates solution to Ax=b using Kaczmarz
    update by cycling through the rows of A.
    b is constant vector.
    A is the coefficient matrix.
    x is an initial iterate.
    maxIter is the number of cycles over A.
    """
    n = length(b)
    for i = 0:(maxIter*n)-1
        k = rem(i,n)+1
        x = kaczmarzUpdate(b[k],A[k,:],x)
    end
    return x
end


#Randomized Kaczmarz
using Distributions
function randomizedKaczmarz(b, A, x; maxIter=1000)
    """
    Estimates solution to Ax=b using randomized
    Kaczmarz, where probability of sampling a row
    depends on the sum of squares of the row.
    Samples are taken with replacement.
    b is a constant vector.
    A is the coefficient matrix.
    x is an initial iterate.
    maxIter is number of rows sampled.
    """
    n = length(b)
    p = map(λ -> sum(A[λ,:].^2), 1:n)/sum(A.^2)
    dist = Categorical(p)
    indices = rand(dist,maxIter)
    for k in indices
        x = kaczmarzUpdate(b[k],A[k,:],x)
    end
    return x
end


randomizedKaczmarz (generic function with 1 method)

In [5]:
#Examples
printstyled(
"EXAMPLE 4: n=100\n")
n = 100
d = n
A = randn(n,d)
x₊ = randn(d)
b = A*x₊

x₀ = randn(d)
x_cyc = cyclicalKaczmarz(b, A, x₀, maxIter=5)
x_ran = randomizedKaczmarz(b, A, x₀, maxIter=5*n)

println("Cyclical Kaczmarz")
println("\tNumber of Iterations: 5")
println("\tResidual Norm: $(norm(A*x_cyc-b))")
println("Randomized Kaczmarz")
println("\tNumber of Iterations: 5000")
println("\tResidual Norm: $(norm(A*x_ran-b))\n")

x_cyc = cyclicalKaczmarz(b, A, x₀, maxIter=10)
x_ran = randomizedKaczmarz(b, A, x₀, maxIter=10*n)

println("Cyclical Kaczmarz")
println("\tNumber of Iterations: 10")
println("\tResidual Norm: $(norm(A*x_cyc-b))")
println("Randomized Kaczmarz")
println("\tNumber of Iterations: 10000")
println("\tResidual Norm: $(norm(A*x_ran-b))\n")

println("Convergence Factor:")
println("\t$(1 - minimum(svdvals(A))^2/sum(A.^2))")


EXAMPLE 4: n=100
Cyclical Kaczmarz
	Number of Iterations: 5
	Residual Norm: 12.140258724836253
Randomized Kaczmarz
	Number of Iterations: 5000
	Residual Norm: 20.693011006433316

Cyclical Kaczmarz
	Number of Iterations: 10
	Residual Norm: 7.557881103765608
Randomized Kaczmarz
	Number of Iterations: 10000
	Residual Norm: 12.531738909102833

Convergence Factor:
	0.9999995284226155


In [6]:
##Conjugated Solver

#Gram-Schmidt Conjugation
function conjGS(A,V; ϵ = 1e-15)
    """Implements classical Gram-Schmidt
    A-conjugation over a set of vectors
    in the vector V. Returns A-conjugated
    vectors using set in V"""

    normA(x) = sqrt(dot(x,A*x))

    Q = Vector(undef, length(V))

    counter = 1
    for j = 1:length(V)
        if counter == 1
            #Check if zero vector
            normA(V[j]) <= ϵ && continue

            #Otherwise add to Conjugated Set
            Q[1] = V[j]/normA(V[j])

            counter+= 1
        else
            #Project
            proj = map(λ -> dot(Q[λ],A*V[j])*Q[λ] ,1:counter-1)
            a = V[j] - sum(proj)

            #Check if zero vector
            normA(a) <= ϵ && continue

            #Otherwise add to Conjugated Set
            Q[counter] = a/normA(a)

            counter +=1
        end
    end
    return Q
end


conjGS (generic function with 1 method)

In [7]:
printstyled(
    "EXAMPLE 5: Sanity Check\n\n", color=:red)
using LinearAlgebra
A = Matrix{Float64}(I,10,10)
V = map(j -> A[:,j],1:5)

Q = conjGS(A,V)

println("Difference between V and Q: $(norm.(Q-V)) \n\n")

printstyled(
    "EXAMPLE 6: Conjugate of Standard Basis\n\n", color=:red)

using Random
Random.seed!(10)
A = randn(10,10); A = A'*A;
V = map(j -> Matrix{Float64}(I,10,10)[:,j], 1:10)

Q = conjGS(A,V)
Q = hcat(Q...) #Make Columns of Matrix
println("Inner Products: $((Q'*A*Q)[:,1])\n\n")

printstyled(
    "EXAMPLE 7: Failed Gram-Schmidt\n\n", color=:red)
A = Matrix{Float64}(I,3,3)
δ = 1e-8
V = Vector{Float64}[
    Float64[1, δ, δ ],
    Float64[1, δ, 0],
    Float64[1, 0, δ]]

Q = conjGS(A,V)
Q = hcat(Q...)
println("Inner Products: $((Q'*A*Q)) \n\n")


EXAMPLE 5: Sanity Check

Difference between V and Q: [0.0, 0.0, 0.0, 0.0, 0.0] 


EXAMPLE 6: Conjugate of Standard Basis

Inner Products: [1.0, -2.44676e-17, 1.99499e-17, -5.39604e-17, -1.74562e-17, -4.86813e-17, -1.66135e-16, 3.28788e-16, -4.82384e-16, 3.71027e-15]


EXAMPLE 7: Failed Gram-Schmidt

Inner Products: [1.0 -1.0e-8 -1.41421e-8; -1.0e-8 1.0 0.707107; -1.41421e-8 0.707107 1.0] 




In [8]:
#Gram-Schmidt Conjugate 2
function conjGS2(A,V; ϵ = 1e-15)
    """Implements Classical GS conjugation
    twice. (Giraud et al. 2005)
    """

    normA(x) = sqrt(dot(x,A*x))

    Q = Vector(undef, length(V))

    counter = 1
    for j = 1:length(V)
        if counter == 1
            #Check if zero vector
            normA(V[j]) <= ϵ && continue

            #Otherwise add to Conjugated Set
            Q[1] = V[j]/normA(V[j])

            counter+= 1
        else
            #Project
            proj = map(λ -> dot(Q[λ],A*V[j])*Q[λ] ,1:counter-1)
            a = V[j] - sum(proj)

            #Check if zero vector
            normA(a) <= ϵ && continue

            #Otherwise Repeat Projection
            proj = map(λ -> dot(Q[λ],A*a)*Q[λ], 1:counter-1)
            a = a - sum(proj)

            #Add Entry to Conjugated Set
            Q[counter] = a/normA(a)

            counter +=1
        end
    end
    return Q
end

conjGS2 (generic function with 1 method)

In [9]:
printstyled(
    "EXAMPLE 8: Conjugate of Standard Basis\n\n", color=:red)

Random.seed!(10)
A = randn(10,10); A = A'*A;
V = map(j -> Matrix{Float64}(I,10,10)[:,j], 1:10)

Q = conjGS2(A,V)
Q = hcat(Q...) #Make Columns of Matrix
println("Inner Products: $((Q'*A*Q)[:,1]) \n\n")

printstyled(
    "EXAMPLE 9: Failed Gram-Schmidt\n\n", color=:red)
A = Matrix{Float64}(I,3,3)
δ = 1e-8
V = Vector{Float64}[
    Float64[1, δ, δ ],
    Float64[1, δ, 0],
    Float64[1, 0, δ]]

Q = conjGS2(A,V)
Q = hcat(Q...)
println("Inner Products: $((Q'*A*Q)) \n\n")

EXAMPLE 8: Conjugate of Standard Basis

Inner Products: [1.0, -2.44676e-17, 1.99499e-17, -1.46934e-17, -7.73635e-17, 4.62289e-17, -7.53922e-17, 1.37255e-16, -4.12767e-16, -5.84077e-16] 


EXAMPLE 9: Failed Gram-Schmidt

Inner Products: [1.0 3.30872e-24 3.30872e-24; 3.30872e-24 1.0 3.69779e-32; 3.30872e-24 3.69779e-32 1.0] 




In [10]:
#Solver

function conjSolv(A,b,x₀, V; ϵ = 1e-15)
    """Solves Ax=b using A-conjugated directions.
    A is symmetric, positive definite matrix.
    b is an arbitrary coefficient vector.
    x₀ is an initial guess.
    V is a vector of linearly ind. vectors.
    """

    Q = conjGS2(A,V)
    r₀ = A*x₀ - b

    α = r₀'*hcat(Q...)
    x = x₀
    for j = 1:length(α)
        x = x - α[j]*Q[j]
    end
    return x
end

printstyled(
    "EXAMPLE 10: Conjugated Direction Solver\n\n", color=:red)

A = randn(10); A = A'*A;
x_true = randn(10)
b = A*x_true
V = map(j -> Matrix{Float64}(I,10,10)[:,j],1:10)
x₀ = randn(10)

y = conjSolv(A,b,x₀,V)

println("Difference between truth and computed:
    \t $(norm(y - x_true)) ")


EXAMPLE 10: Conjugated Direction Solver

Difference between truth and computed:
    	 1.1043650758119165e-15 


In [11]:
# Conjguated Gradient Solver
function conjGraSolv(A,b,x₀; ϵ = 1e-15)
    """Implementation of CG method.
    A is symmetric positive definite
    b is a constant vector.
    x0 is an initial guess."""

    #Update Iterate & Residual
    x = x₀
    r = b - A*x;
    p = r;
    rsold = r' * r;

    for i = 1:length(b)
        Ap = A * p;
        alpha = rsold / (p' * Ap);
        x = x + alpha * p;
        r = r - alpha * Ap;
        rsnew = r' * r;
        if sqrt(rsnew) < 1e-15
            println("Breaking at iteration $i")
              break;
        end
        p = r + (rsnew / rsold) * p;
        rsold = rsnew;
    end
    return x
end

printstyled(
    "EXAMPLE 11: Conjugated Gradient Solver\n\n", color=:red)

using LinearAlgebra
A = randn(100); A = A'*A .+ Diagonal(10*rand(100));
x_true = randn(100)
b = A*x_true
x₀ = randn(100)

y = conjGraSolv(A,b,x₀)

println("Difference between truth and computed:
    \t $(norm(y - x_true)) ")


EXAMPLE 11: Conjugated Gradient Solver

Difference between truth and computed:
    	 5.684208002380652e-12 
